# Part 3: EM implementation

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
def read_data():
    with open('sequence.padded.txt') as handle:
        # I dont think this is the actual sequences we should be using
        # put using as placeholder for now.
        return [s.strip() for s in handle.readlines()]

data = read_data()
data

['ATACCCCTGGCTGGGTCATGGTGACCTGGAGGAAGCGT',
 'CATATATGGCCAGGGTCAGTGTGACCTCCATTTCCCAT',
 'AGCAGCTGGCCTGGGTCACAGTGACCTGACCTCAAACC',
 'AGGCTGTGTACAAGGTCAGAGTGACCTCTAGAAGCTCT',
 'TACTCTAGTTCCAGGTCATGGTGACCTGTGAAAAATCT',
 'AGGACTGTTTCAAGGTCACGGTGACCCTCGTGGGCTGT',
 'GCAGGAAGTTTTGGGTCACGGTGACCTCTAGTTGTTGA',
 'CAAGTGCTTCAAAGGTCATGGTGCCCTGGGGCCGAGAG',
 'ACCAACATGGCAGGGTCAAGTTGACCTCCCTGGCCACT',
 'TCTCTCTCTAGTAGGTCATGGTGACCTGTACACATTAT',
 'TCAGACCACAGAGGGTCAAGGTGACCTGAGAGATCAGT',
 'AGGCAATTCACTAGGTCAGGATGCCCTGGGGCAACAGT',
 'TAGTCCTGAAAAGGGTCATGTTGACCTGATTGTCATGT',
 'ATTAACTCTTCTAGGTCAGTGTGACCTAAACTCATCGG',
 'GGACAATTATTGGGGTCACGGTGACCTGCCTGTTTCAG',
 'GGTCCATAATATAGGTCATGTTGACCTGGGACAACTGG',
 'CTCCAGGAGCAGGGGTCAGGGTGACCTCCAGCTCCTCA',
 'GAGCCCATCTCTGGGTCATGTTGCCCTCTTACAGCACA',
 'TGGGTTAAACCTGGGTCATGTTGACCTAGATACATCTC',
 'GTGACATCCCCAGGGTCAAAGTGCCCTGAGTCTGGAGA',
 'GCCTTCTAGGTCAGCATGACCTGGTCCTCAGAGGGGGG',
 'GGCAATGAATCAAGGTCAGGCTAACCTGGCTTACTGCA',
 'CCTACTAGCCCTGGGTCAACGTGCCCTGTAAGAGCATG',
 'GGCGCAGCC

Create matrix $X_{i,j,p,k}$ using one-hot encoding scheme.

In [112]:
seq_length = len(data[0])
motif_length = 8
number_motifs = seq_length - motif_length + 1
X = np.zeros((len(data), number_motifs, motif_length, 4))

def nuc_to_one_hot(nuc):
    # Convert nucleotide to the index in one hot encoded array
    # that should be hot (==1)
    upper_nuc = nuc.upper()
    mapping = {'A': 0, 'T': 1, 'G': 2, 'C': 3}
    return mapping[upper_nuc]

j_p = []
for i in range(seq_length):
    for j in range(number_motifs):
        for p in range(motif_length):
            nuc = data[i][j+p]
            k_hot = nuc_to_one_hot(nuc)
            X[i][j][p][k_hot] = 1.0

assert X.sum() == seq_length * number_motifs * motif_length

Randomly initialize the model parameters.

In [113]:
def init_EM(seq_length, motif_length):
    number_motifs_per_sequence = seq_length - motif_length + 1
    lambda_j = np.random.uniform(0, 1, size=(number_motifs_per_sequence,))
    lambda_j_norm = lambda_j / lambda_j.sum()
    psi_0 = np.random.uniform(0, 1, size=(4, motif_length))
    psi_1 = np.random.uniform(0, 1, size=(4, motif_length))
    psi_0 = (psi_0.T/psi_0.sum(axis=1)).T  # https://stackoverflow.com/questions/16202348/numpy-divide-row-by-row-sum
    psi_1 = (psi_1.T/psi_1.sum(axis=1)).T
    
    return lambda_j_norm, psi_0, psi_1

In [116]:
lambda_j, psi_0, psi_1 = init_EM(seq_length, motif_length)

## E step

![](e.png)

First calculate the numerator of the equation.

In [124]:
def e_numerator(i, j, X, lambda_j, psi_0, psi_1):
    # i = current sequence
    # j = current j index
    # X = Data
    psi_1_term = (X[i][j] * psi_1.T)
    # remove zero terms this is in lieu of having exponent X_{i,j,p,k}
    # whcih would cause 0 encoded values (cold values) to have
    # value of 1 and therefore not contribute to the product
    psi_1_term = psi_1_term[psi_1_term != 0]
    # take product of all remaining terms (these are probibities seeing
    # the bases in the given motif in their given positions given they
    # are in the TFBS)
    psi_1_term = np.log(psi_1_term).sum()
    
    # now need to get product of all other motifs (j' != j) but assuming they are
    # not the TFBS (psi^0). 
    psi_0_term = X[i][np.arange(len(X[i]))!=j] * psi_0.T
    psi_0_term = psi_0_term.flatten()
    psi_0_term = np.log(psi_0_term[psi_0_term != 0]).sum()
    return np.log(lambda_j[j]) + psi_0_term + psi_1_term  # log of products is sum of logs

In [119]:
numerator(0, 0, X, lambda_j, psi_0, psi_1)

-604.4315015640672

In [125]:
def e_denominator(i, j, X, lambda_j, psi_0, psi_1):
    # Get all lambda values that are not j
    lambda_j_prime = np.delete(lambda_j, j)
    sum_terms = []
    j_prime_indexes = np.delete(list(range(len(lambda_j))), j)
    for j_prime in j_prime_indexes:
        psi_1_term = X[i][j_prime] * psi_0.T
        psi_1_term = psi_1_term[psi_1_term != 0].flatten().prod()
        psi_0_term = X[i][np.arange(len(X[i]))!=j_prime] * psi_0.T
        psi_0_term = psi_1_term[psi_1_term != 0].prod()
        sum_terms.append(
            psi_1_term * psi_0_term * lambda_j[j_prime]
        )
    return np.log(sum(sum_terms))

In [121]:
denominator(0, 0, X, lambda_j, psi_0, psi_1)

-32.26328611566109

In [141]:
def e_step_ij(i, j, X, lambda_j, psi_0, psi_1):
    # Calculate P(C_{i} = j | X_{i}, \theta) for 1 i and j value
    return e_numerator(i, j, X, lambda_j, psi_0, psi_1) - e_denominator(i, j, X, lambda_j, psi_0, psi_1) 

Function to do one iteration of the I step.

In [145]:
def e_step(X, lambda_j, psi_0, psi_1):
    posts = []
    for i in range(X.shape[0]):  # number of sequences
        posts.append(
            [e_step_ij(i, j, X, lambda_j, psi_0, psi_1) for j in range(X.shape[1])]
        )
    return np.array(posts)

e_step_1 = e_step(X, lambda_j, psi_0, psi_1)
e_step_1[1]

array([-561.62499968, -561.7284384 , -566.68139075, -559.75908395,
       -569.81150711, -564.11426938, -571.00151514, -564.9640083 ,
       -556.81109116, -557.99471054, -564.32950531, -569.30222129,
       -561.25281779, -563.72549226, -555.54360909, -567.83317061,
       -566.18347438, -568.888207  , -556.64644839, -569.4309473 ,
       -555.9208588 , -570.94257976, -568.38596906, -565.99833547,
       -562.45530041, -560.59520836, -566.90519787, -562.83709335,
       -564.79951559, -565.6166644 , -568.49565769])

Calculate ELBO. What exactly is ELBO here?

#### ELBO lecture notes

- 

In [ ]:
def loglikelihood(XXss, lambda_j, psi_0, psi_1, posteriors):
    

In [150]:
sum(np.e ** e_step_1[2])

6.88684287167941e-245

## M step

Quon says $\boldsymbol{E}[C_{i,j}] = P(C_{i} = j | X_{i}, \theta)$

He also gives what $\boldsymbol{E}[C_{i,j}] = P(C_{i} = j | X_{i}, \theta)$ equals to in the E step (shown in the image below.)

### $\lambda_{j}$

Take sum of all values at each index $i$ over vector $C_{i, j}$ which would store prob each $j$ (each motif) being the transcription factor binding site and divide this value by the number of sequences $N$.
- Would this not always just sum to 1? And therefore $\lambda_{j}$ is basically fixed at 1 over the number of sequences?

In [169]:
e_step_1.shape

(357, 31)

In [170]:
e_step_1[0]

array([-572.16821545, -580.7835557 , -577.38610676, -576.4721679 ,
       -570.11963541, -571.41823003, -571.78661016, -582.11858281,
       -569.35081316, -567.17825307, -572.22665658, -581.58766499,
       -571.50237156, -575.87204121, -567.41126423, -574.52390012,
       -576.24121983, -575.0131689 , -576.78227197, -572.02878225,
       -567.38997154, -581.28441579, -576.08230422, -574.26195648,
       -572.64333978, -571.28948864, -577.40812545, -571.41794428,
       -569.90676362, -573.42478895, -568.176371  ])

In [ ]:
e_step_1

In [174]:
def lambda_j_m_step(E_Cij, number_seqs):
    return E_Cij.sum(axis=0) / number_seqs

In [171]:
new_lambda_j = lambda_j_m_step(e_step_1, X.shape[0])

In [172]:
new_lambda_j

array([-62.71691577, -63.05779531, -63.31498319, -64.76568963,
       -63.10450845, -63.14917784, -63.88537224, -63.52601493,
       -62.4799424 , -63.25383101, -64.56375275, -64.04239571,
       -63.01064996, -64.48868862, -61.98046732, -63.36127541,
       -66.61330252, -66.37574594, -62.68994602, -62.92390036,
       -62.51928642, -65.01503091, -63.63956944, -63.73426561,
       -63.55859655, -63.12894706, -63.42997417, -62.76863772,
       -63.44980247, -63.52538458, -63.18981867])

In [173]:
assert new_lambda_j.shape == lambda_j.shape

### $\psi^{1}_{p, k}$

- How does taking a sum over $i$ for $C_{i, j}$look compared to taking a sum over $i$ and $j$?

Product of indicator variables for a given motif (For example the matrix at `X[0][0]`) and the expectation at that motif calculated during the E step. Then take a sum overall all those values and divide by the number of sequences.

In [175]:
(X[0][0][0][0]*e_step_1[0][0]) / N # this would product one cell in the psi matrix need to iterate through p and Ks

-1.6027120880907733

In [176]:
def psi_1_m_step(E_Cij, psi_1, num_seqs):
    for p in range(psi_1.T.shape[0]):
        for k in range(psi_1.T.shape[1]):
            products = []
            for i in range(X.shape[0]):
                for j in range(X.shape[1]):
                    products.append(X[i][j][p][k] * E_Cij[i][j])
            psi_1[k][p] = sum(products) / num_seqs
    return psi_1

psi_1_m_step(e_step_1, psi_1, X.shape[0]).shape

(4, 8)

### $\psi^{0}_{p, k}$

Seems pretty much like other $\psi$ but add some subtractions and change the denominator to the number of sequences times the number of possible motifs.

In [168]:
seq_len = 100
def psi_0_m_step(E_Cij, psi_0, seq_len, num_seqs):
    for p in range(psi_1.T.shape[0]):
        for k in range(psi_1.T.shape[1]):
            products = []
            for i in range(X.shape[0]):
                for j in range(X.shape[1]):
                    products.append(1 - X[i][j][p][k] * rand[i][j])
        psi_1[k][p] = sum(products) / ((seq_len - X.shape[1] + 1 - 1) * num_seqs)
    
    return psi_1

psi_0_m_step(e_step_1, psi_0, seq_len, X.shape[0])

array([[-4.23607898e+02, -4.20529241e+02, -4.26709246e+02,
        -4.27233861e+02, -4.27895316e+02, -4.23155964e+02,
        -4.23142528e+02, -4.20013611e+02],
       [-4.55653290e+02, -4.48071800e+02, -4.49344839e+02,
        -4.44374676e+02, -4.46188364e+02, -4.48676340e+02,
        -4.35623737e+02, -4.36874457e+02],
       [-5.13292563e+02, -5.17618903e+02, -5.10672147e+02,
        -5.14357574e+02, -5.12092142e+02, -5.14136138e+02,
        -5.27983553e+02, -5.40014776e+02],
       [ 4.43182475e-01,  4.43509192e-01,  4.43073123e-01,
         4.43163578e-01,  4.43034622e-01,  4.43192189e-01,
         4.43106579e-01,  4.43459694e-01]])

In [177]:
X[0][0]

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]])

In [178]:
X[0][0] * psi_1.T

array([[-423.60789849,   -0.        ,   -0.        ,   -0.        ],
       [  -0.        , -448.07180045,   -0.        ,   -0.        ],
       [-426.7092458 ,   -0.        ,   -0.        ,   -0.        ],
       [  -0.        ,   -0.        ,   -0.        , -483.94149548],
       [  -0.        ,   -0.        ,   -0.        , -483.73178397],
       [  -0.        ,   -0.        ,   -0.        , -483.9391633 ],
       [  -0.        ,   -0.        ,   -0.        , -483.15778753],
       [  -0.        , -436.87445744,   -0.        ,   -0.        ]])